In [62]:
import pandas as pd
from datetime import datetime, timedelta
import pytz
import requests 
import json

### Helper Code

In [69]:
key = "A_vXSwpuQ4hyNRj_8Rlw1WwVDWGgHbjp"

# # Set display options
# pd.set_option('display.max_rows', None)  # No limit on the number of rows displayed
# pd.set_option('display.max_columns', None)  # No limit on the number of columns displayed
# pd.set_option('display.width', None)  # Adjust display width for wide columns
# pd.set_option('display.max_colwidth', None)  # Display full content of each column

def convert_timestamp_est(timestamp):
    # Create a UTC datetime object from the timestamp
    utc_datetime = datetime.utcfromtimestamp(timestamp).replace(tzinfo=pytz.utc)
    # Define the EST timezone
    est_timezone = pytz.timezone('America/New_York')
    # Convert the UTC datetime to EST
    est_datetime = utc_datetime.astimezone(est_timezone)
    return est_datetime


def polygon_call_stocks(contract, from_stamp, to_stamp, multiplier, timespan, open_date):
    trading_hours = ['09:30:00', '16:00:00']
    try:
        payload={}
        headers = {}
        url = f"https://api.polygon.io/v2/aggs/ticker/{contract}/range/{multiplier}/{timespan}/{from_stamp}/{to_stamp}?adjusted=true&sort=asc&limit=50000&apiKey={key}"
        response = requests.request("GET", url, headers=headers, data=payload)
        res_df = pd.DataFrame(json.loads(response.text)['results'])
        res_df['t'] = res_df['t'].apply(lambda x: int(x/1000))
        res_df['date'] = res_df['t'].apply(lambda x: convert_timestamp_est(x))
        res_df['year'] = res_df['date'].apply(lambda x: x.year)
        res_df['month'] = res_df['date'].apply(lambda x: x.month)
        res_df['time'] = res_df['date'].apply(lambda x: x.time())
        res_df['hour'] = res_df['date'].apply(lambda x: x.hour)
        res_df['minute'] = res_df['date'].apply(lambda x: x.minute)
        res_df['dt'] = res_df['date'].apply(lambda x: datetime(x.year, x.month, x.day,x.hour,x.minute))
        res_df = res_df[(res_df['time'] >= datetime.strptime(trading_hours[0], '%H:%M:%S').time()) & (res_df['time'] <= datetime.strptime(trading_hours[1], '%H:%M:%S').time())]
        res_df = res_df[res_df['dt'] >= datetime.strptime(open_date, '%Y-%m-%d %H:%M')]
            
        return res_df
    except Exception as e:  
        print(e)
        return pd.DataFrame()
    
def options_min_max_value(row):
    try:
        to_dt = datetime.strptime(row['dt_x'], '%Y-%m-%d') + timedelta(days=1)
        to_stamp = to_dt.strftime('%Y-%m-%d')
        options_aggregates = polygon_call_stocks(row['option_symbol'], row['dt_x'], to_stamp, '30', 'minute',row['open_trade_dt'])
        max_price = options_aggregates['h'].max()
        min_price = options_aggregates['l'].min()
        open_price = options_aggregates.iloc[0]['o']
    except Exception as e:
        print(options_aggregates)
        max_price = 0
        min_price = 0
        open_price = 0
    return max_price, min_price, open_price


In [80]:
model_df = pd.read_csv('combined_model_profits.csv')

In [81]:
test = model_df
model_df.head(3)

,Unnamed: 0,price_change,pct_gain,total_gain,open_trade_dt,close_trade_dt,max_gain_before,max_gain_after,option_symbol,max_value_before_date,max_value_after_date,max_value_before_idx,max_value_after_idx,sell_code,quantity,position_id,underlying,modeling_strategy,gain,value_diff_before,value_diff_after,open_dt,close_dt,close_dw,open_dw,days_held,dt_x,hour_x,probabilities,label,predictions,symbol,dt_y,strategy,hour_y
0,0,-0.89,-40.271493,-89.0,2023-01-03 10:00,2023-01-03 11:15,0.027149,0.040724,O:QQQ230105C00267000,2023-01-03 10:00,2023-01-04 11:30,0,31.0,Breached floor pct,1,QQQ-2023-01-03-10-CDBFC1D,QQQ,CDBFC1D,-0.402715,0.429864,0.443439,2023-01-03 10:00:00,2023-01-03 11:15:00,1,1,1,2023-01-03,10,0.635010,0,1,QQQ,2023-01-03,CDBFC1D,10
1,1,-0.80,-36.199095,-160.0,2023-01-03 10:00,2023-01-03 11:15,0.085973,0.565611,O:SPY230105C00384000,2023-01-03 10:00,2023-01-04 11:45,0,33.0,Breached floor pct,2,SPY-2023-01-03-10-CDBFC1D,SPY,CDBFC1D,-0.361991,0.447964,0.927602,2023-01-03 10:00:00,2023-01-03 11:15:00,1,1,1,2023-01-03,10,0.692993,1,1,SPY,2023-01-03,CDBFC1D,10
2,2,-0.66,-42.857143,-66.0,2023-01-03 10:00,2023-01-03 11:45,0.168831,0.474026,O:IWM230106C00176000,2023-01-03 11:00,2023-01-04 11:45,4,33.0,Breached floor pct,1,IWM-2023-01-03-10-CDBFC1D,IWM,CDBFC1D,-0.428571,0.597403,0.902597,2023-01-03 10:00:00,2023-01-03 11:45:00,1,1,1,2023-01-03,10,0.605305,1,1,IWM,2023-01-03,CDBFC1D,10


In [82]:
results = test.apply(options_min_max_value, axis=1, result_type='expand')

In [83]:
results.columns = ['max_price', 'min_price','open_price']
test = pd.concat([test, results], axis=1)

In [84]:
test.to_csv('all_samples.csv', index=False)

In [85]:
print(len(test))

15043


### Analysis of Options Outcomes

In [73]:
test['max_profit']= test['max_price'] - test['open_price']
test['min_profit']= test['min_price'] - test['open_price']

KeyError: 'max_price'